In [1]:
import h5py
import numpy as np
from timeit import timeit
import posixpath
from functools import partial
import time, datetime

## Ch. 1 & 2: Introduction

In [ ]:
# 1st example
temps = np.random.random(1024) # time series data as array (our dataset)
dt_temp = 10.0 # sampling rate
t_start = 1375204299
station = 15
winds = np.random.random(2048) # second dataset
dt_wind = 5.0 # different sampling rate

In [ ]:
f = h5py.File("weather.h5","w") # careful, 'w' mode overwrites existing file
f["/15/temperature"] = temps # dataset
f["/15/temperature"].attrs["dt"] = dt_temp # attributes (dict style)
f["/15/temperature"].attrs["t_start"] = t_start
f["/15/winds"] = winds
f["/15/winds"].attrs["dt"] = dt_wind
f.close()

Here we declare the output file. Option "a" is the default.

(Read/write if exists, create otherwise, but don't overwrite anything)  

Once we've run the above block, we can't run it again, b/c things would be overwritten.

Also note, groups are created by using the "/", or we could have used `f.create_group("name")`

In [ ]:
f = h5py.File("weather.h5")
print(f.keys()) # this should almost be called "groups"
print(f["/15"].keys()) # give the subgroups
f.close()

In [ ]:
# "context manager" - closes file when the task is done
with h5py.File("weather.h5") as f:
    print(f.keys())

In [ ]:
# h5py has a funny way of iterating through groups,
# you have to pass 'visititems' a function
def print_attrs(name, obj):
    print(name)
    for key, val in obj.attrs.items():
        print("    %s: %s" % (key, val))

f = h5py.File('weather.h5','r')
f.visititems(print_attrs)

Now look at a few different "file drivers".

NOTE: In this notebook I am using "w" a lot because I don't care about deleting/recreating these files.  

Check this: http://docs.h5py.org/en/stable/high/file.html#

In [ ]:
# default: auto for the system
f = h5py.File("t1.h5","w") 

# load the file into memory
f = h5py.File("t2.h5","w", driver="core") 

# load into memory, but create an on-disk file where
# the file image is saved when closed.
# also tells hdf5 to load any existing image from disk
# when you open the file.
f = h5py.File("t3.h5","w", driver="core", backing_store=True) 

# split a file into multiple "images" w/ a declared max size.
# not working here for some reason.
# f = h5py.File("t4.h5","w", driver="family", memb_size=1024**3)

# mpio driver for Parallel HDF5 (see ch 9)
# f = h5py.File("t5.h5", "w", driver='mpio')

f.close()

In [ ]:
# h5 files have a "user block" @ the beginning, where you
# can store arbitrary data.  You can't have the file open
# in HDF5 while writing to the user block.
f = h5py.File("userblock.h5", "w", userblock_size=512)

with open("userblock.h5", "r+") as f: # open as regular python file
    # f.write("a"*512)
    f.write("oh hai i'm clint")
    
# i can't figure out how to actually READ it though ...
with open("userblock.h5") as f:
    f.readline() # gives an error

## Chapter 3: Working With Datasets

In [ ]:
# dataset basics
with h5py.File("test.h5", "w") as f:
    arr = np.ones((5,2))
    f["my dataset"] = arr
    
    # this is a "proxy" object that lets you read/write to the
    # underlying HDF5 dataset on disk.  
    # but it's LIKE a numpy array in many respects.
    dset = f["my dataset"]
    print(dset)
    print(dset.dtype)
    print(dset.shape)
    
    # read the dataset into a np array w/ the Ellipsis.
    # another way of saying it:
    # "slicing INTO a dataset object returns a numpy array"
    # this is where HDF5 actually reads the data from disk.
    out = dset[...]
    print(out)
    print(type(out))
    
    # update a portion of the dataset
    dset[1:4, 1] = 2.0
    print(dset[...])
    
    # create some empty datasets
    dset2 = f.create_dataset("test2", (10, 10))
    dset3 = f.create_dataset("test3", (10, 10), dtype=np.complex64)
    print(dset3)

In [ ]:
# make an empty dataset that is really big (preallocated)
# and only write to a small part of it.

# NOTE: the book is outdated, apparently.
# you have to turn on chunking for h5py
# to not allocate space for the entire file.
# https://stackoverflow.com/questions/45145389/size-on-disk-of-a-partly-filled-hdf5-dataset
f = h5py.File("test.h5","w")

# auto-determine chunk size
# ds = f.create_dataset("bigds", (1024**3,), dtype=np.float32, chunks=True)

# set chunk size manually (16384 floats = 64 KB)
ds = f.create_dataset("bigds", (1024**3,), dtype=np.float32, chunks=(2**14,) )
print(ds.chunks)

# chunk size has performance implications. 
# It’s recommended to keep the total size of your chunks 
# between 10 KiB and 1 MiB, larger for larger datasets. 
# Also keep in mind that when any element in a chunk is 
# accessed, the entire chunk is read from disk.

ds[0:1024] = np.arange(1024)
f.flush()
f.close()

In [ ]:
# use float64s in memory to minimize rounding error, 
# but store as float32s to save space
bigdata = np.ones((100,1000))
# print(bigdata.dtype, bigdata.shape)

with h5py.File("big1.h5",'w') as f:
    # f["big"] = bigdata # 783K
    f.create_dataset("big", data=bigdata, dtype=np.float32) # 393K

In [ ]:
%%bash 
ls -lh big1.h5

if we had a single-precision float dataset 
on disk, but wanted to read it in as double-precision,
we would have to be careful b/c it might be more than memory
or we might want to do the type conversion on the fly

In [ ]:
big_out = np.empty((100,1000), dtype=np.float64)

with h5py.File("big1.h5") as f:
    dset = f["big"]
    
    # method 1: preallocated numpy array of the DESIRED type.  
    dset.read_direct(big_out)
    
    # method 2 : read with "astype" - hdf5 converts on the fly
    with dset.astype('float64'):
        out = dset[0,:] # read by slicing
    
print(big_out[:3,:3], big_out.dtype, out.dtype)

In [ ]:
# be careful about how you slice!  
# it can impact performance.

with h5py.File("big1.h5") as f:
    dset = f['big']
    out = dset[0:10, 20:70]
    
f = h5py.File("big1.h5")
dset = f["big"]

# # the slow way (> 10 seconds!)
# for ix in range(100):
#     for iy in range(1000):
#         val = dset[ix, iy]
#         if val < 0 : 
#             dset[ix, iy] = 0 # trim 

# the fast way (done ~ instantly)
for ix in range(100):
    val = dset[ix, :] # read a whole row into ndarray
    val[ val < 0 ] = 0 # trim
    dset[ix, :] = val # write back to file
            
f.close()

# moral of the story: 
# writing to a Dataset one element (or even a few elements)
# at a time, is a great way to get poor performance.

In [ ]:
f = h5py.File("test2.h5", "w")

# multidimensional slicing
dset = f.create_dataset('4d', shape=(100,80,50,20))
print(dset[90,...,0].shape)
print(dset[90,...].shape)

# handling scalar datasets

# method 1 - a 1d numpy array
dset = f.create_dataset('1d', shape=(1,), data=42)
print(dset.shape)
print(dset[0], dset[...]) # note the different outputs

# method 2 - empty tuple, shape is (), indexing doesn't work
dset = f.create_dataset('0d', data=42)
print(dset.shape)
# print(dset[0]) # fails
print(dset[...], type(dset[...])) # this returns an ARRAY 
print(dset[()], type(dset[()])) # this wacky thing gives you the value

f.close()

boolean indexing is great for np arrays, `idx = np.where(blahblahblah)` and we can also do it directly on HDF5 Dataset objects:

In [ ]:
data = np.random.random(10)*2 - 1
print(data.shape)

f = h5py.File("test2.h5", "w")
ds = f.create_dataset("randoms", data=data)

# do the indexing on the dataset, using the np array
ds[data < 0] = 0 
print(ds[...])

# note from the book:
# for very large indexing expressions with lots of True values,
# it may be faster to, for example, modify the data on the Python
# side and write the dataset o
ut again.  If you suspect a slowdown
# it's a good idea to test this.

# modify some elements in place (array is same length as slice)
ds[data < 0] = -1 * data[data<0]
print(ds[...])

# can also specify given coordinate lists.
# NOTE: this is much more efficient than Boolean masking 
# for large datasets
print(ds[[1,2,7]])

f.close()

In [ ]:
# automatic broadcasting

# when we make slicing assignment where the number of elements
# on the LHS and RHS are not equal, numpy and hdf5 use "broadcasting":
# dset[data<0] = 0
# "used judiciously, it can give your application a performance boost"

f = h5py.File("big1.h5")
dset = f["big"]

# suppose we want to copy the trace at dset[0, :], 
# and overwrite all the others in the file.

# method 1 (slow)
# we have to write the loop, get the bc's right, and perform 100 slices
data = dset[0,:]
for i in range(100):
    dset[i,:] = data
    
# method 2 (fast, w/broadcasting)
dset[:,:] = dset[0,:]

# RHS is (1000,) LHS is (100,1000).
# Since the last dimensions match, h5py repeatedly copies the data
# across all 100 remaining indies. 
# There's only one slicing operation!

f.close()

In [ ]:
# reading directly into an existing array
# automatically performs type conversion

f = h5py.File("big1.h5")
dset = f['big']

# ex. 1 (repeated) - read float32 data into a float64 ndarray
print(dset.dtype)
out = np.empty((100,1000), dtype=np.float64)
dset.read_direct(out)

# ex. 2
# suppose we wanted to read the first time trace [0,:]
# and deposit it into the out array at out[50,:]
# we can select both the source and destination:
dset.read_direct(out, source_sel=np.s_[0,:], dest_sel=np.s_[50,:])

# here np._s is a thing that takes slices and returns a np "slice" object

# you don't have to match the shape of the output array to the dataset:

# method 1:
out = dset[:,0:50]
means = out.mean(axis=1)

# method 2, with read_direct:
# this is faster when the shapes of the arrays are really huge
out = np.empty((100,50), dtype=np.float32)
dset.read_direct(out, np.s_[:,0:50]) # dest_sel can be omitted
means = out.mean(axis=1)

f.close()

In [ ]:
# "endian-ness" : relating to how multi-byte numbers are represented.
# if we wanted to store a 4-byte floating point number:
# little-endian : stores the LEAST significant byte first
# big-endian : stores the MOST significant byte first

a = np.ones((1000,1000), dtype='<f4') # little-endian 4-byte float
b = np.ones((1000,1000), dtype='>f4') # big-endian 4-byte float

%timeit a.mean
%timeit b.mean

# huh.  the book says big-endian is a factor 2 slower, 
# but they're pretty much identical here. 
# i guess stuff's been updated

# to convert to "native" endian-ness for your system:
# 1. use read_direct w/ a natively formatted array you create yourself
# 2. use the 'astype' context manager
# 3. convert the numpy array in place:
c = b.view("float32")
c[:] = b
b = c
%timeit b.mean

In [ ]:
# resizing datasets

# datasets have a SHAPE and TYPE.  
# TYPE is unchangeable, but SHAPE is not.

f = h5py.File("test3.h5","w")

dset = f.create_dataset('fixed', (2,2)) # a 4-element dataset
print(dset.shape, dset.maxshape)

# huh.  so we have to set dset.maxshape
dset = f.create_dataset('resizable', (2,2), maxshape=(2,2))
dset.resize((1,1))
print(dset.shape)

# what if you don't know when you create the dataset how big it should be?
# like maybe during data acquisition from a digitizer?
dset = f.create_dataset('unlimited', (2,2), maxshape=(2,None))
dset.resize((2, 2**30))
print(dset.shape)

# note that you can mark as many axes you want as unlimited.

# but you CAN'T change the total number of axes:
# dset.resize((2,2,2)) # throws error

f.close()

In [ ]:
# shuffling data w/ resize

print("numpy side:")
a = np.array([[1,2],[3,4]])
print(a)
a.resize((1,4))
print(a)
a.resize((1,10)) # new elements are initialized to 0
print(a)

# hdf5 side: no reshuffling is ever performed

f = h5py.File("test3.h5","w")

dset = f.create_dataset('sizetest', (2,2), dtype=np.int32, maxshape=(None,None))

print("hdf5 side")
dset[...] = [[1,2],[3,4]]
dset.resize((1,4))
print(dset[...])
dset.resize((1,10))
print(dset[...])
print("whoops, applying numpy resizing to datasets caused us to lose data!")

f.close()

In [ ]:
# appending to a dataset with "resize"

f = h5py.File("test3.h5", "w")

dset1 = f.create_dataset("timetraces", (1,1000), maxshape=(None, 1000))

def add_trace_1(arr):
    # every time a new 1000-element array is added,
    # the dataset is simply expanded by a single entry
    dset1.resize((dset1.shape[0]+1, 1000))
    dset1[-1,:] = arr
    
# be careful, if the number of resize calls is equal to the
# number of insertions, this doesn't scale well.

# alternatively, keep track of the # insertions and then "trim"
# the dataset when done:

dset2 = f.create_dataset('timetraces2', (5000,1000), maxshape=(None,1000))

ntraces = 0
def add_trace_2(arr):
    global ntraces
    dset2[ntraces,:] = arr
    ntraces += 1

def done():
    dset2.resize((ntraces,1000))


f.close()

## Chapter 4: Chunking and Compression

By default, all but the smallest HDF5 datasets use contiguous storage.
The data in your DS is flattened to disk using the same rules as numpy.

In [ ]:
# consider a set of 100 images:
f = h5py.File("imagetest.h5","w")

dset = f.create_dataset("Images", (100,480,640), dtype='uint8')

# a contiguous dataset stores the images on disk,
# one 640-element "scanline" after another

image = dset[0,:,:]
print(image.shape)

# what if we wanted to only access the first 64x64 tile?
tile = dset[0,0:64,0:64]

# if we looped over the images, we wouldn't be able to access
# the tiles as a contiguous block, which is slow.
# %timeit dset[:,0:64,0:64]

# a better approach: use chunking
dset = f.create_dataset("chunked", (100,480,640), dtype="i1", chunks=(1,64,64))
print(dset.chunks)
# %timeit dset[:, 0:64, 0:64]

# auto chunking w/o specifying chunk size
dset = f.create_dataset("Images2", (100,480,640), "f", chunks=True)
print(dset.chunks)

# NOTE: chunks bigger than 1MB can't be loaded into the fast cache

# TIP: you should only do chunking in cases when you know for sure
# your dataset will be accessed in a way that's likely to be inefficient
# with either contiguous storage or an auto-guessed chunk shape.

# for my stuff, maybe a good "chunk" would be a waveform, 
# i.e. chunks=(1,2016) or whatever it is
# OR, maybe it would be the pygama "block size", so chunks=(3000,2016),
# but maybe that would be more than 1MB.  idk.  try it out sometime.

f.close()

In [ ]:
# resizing datasets (revisiting chunking)

f = h5py.File("resizing.h5", "w")

# declare two datasets, both expandable, w/ differing initial sizes
dset1 = f.create_dataset("timetraces1", (1, 1000), maxshape=(None, 1000))
dset2 = f.create_dataset("timetraces2", (5000, 1000), maxshape=(None, 1000))

def add_trace_1(arr):
    # Add a trace to the ds, expanding it as necessary
    dset1.resize((dset1.shape[0]+1, 1000))
    dset1[-1,:] = arr
    
ntraces = 0
def add_trace_2(arr):
    # Add a trace to the ds, keeping count of the # traces written
    global ntraces
    dset2[ntraces,:] = arr
    ntraces +=1
    
def done():
    # after all calls to add_trace_2, trim the ds to size
    dset2.resize((ntraces,1000))
    
def setup():
    # re-init both datasets for the tests
    global data, N, dset1, dset2, traces
    data = np.random.random(1000)
    N = 10000 # num iterations
    dset1.resize((1,1000))
    dset2.resize((10001,1000))
    ntraces = 0
    
def test1():
    # add N traces to the first ds
    for ix in range(N):
        add_trace_1(data)
        
def test2():
    # add N traces to the second ds, then trim it
    for ix in range(N):
        add_trace_2(data)
    done()

# different from the book again.  method 2 is faster for me
# print(timeit(test1, setup=setup, number=1)) # 1.89
# print(timeit(test2, setup=setup, number=1)) # 1.69

# print(dset1.chunks) # same as book
# print(dset2.chunks) # different from book

# so chunk shape is partly determined by the INITIAL SIZE of the dataset

# try it again with a manual chunk shape:

dset1 = f.create_dataset("timetraces3", (1,1000), maxshape=(None,1000), chunks=(1,1000))
dset2 = f.create_dataset("timetraces4", (5000,1000), maxshape=(None,1000), chunks=(1,1000))

# ok, method 2 is faster again, as it should be
print(timeit(test1, setup=setup, number=1)) # 1.83
print(timeit(test2, setup=setup, number=1)) # 1.55

    
f.close()

In [ ]:
# filters and compression
# the filters work on CHUNKS in hdf5

f = h5py.File("filters.h5","w")

# gzip compression filter
dset = f.create_dataset("BigDataset",(1000,1000),dtype='f',compression='gzip')
print(dset.compression)
print(dset.compression_opts) # for gzip, this is compression level
print(dset.chunks) # 63*125*(4 bytes) = 30 KiB blocks for the compressor

# compression is transparent (you don't notice it)
dset[...] = 42.0
print(dset[0,0])

# other lossless compressors are LZF (python only), SZIP (proprietary, don't use)
# also BLOSC, BZIP2, etc.
# "it's also rare for an application to spend most of its time compressing
# or decompressing data, so try not to get carried away with speed testing."

# SHUFFLE filter: use in conjunction w/ gzip to improve compression
# for many datasets, most of the entropy occurs in a few bytes

# FLETCHER32 filter: creates a checksum for each chunk, recorded in the
# chunk's metadata.  When the chunk is read, we compare to the checksum
# to check for any losses.

# NOTE:  I would guess that for LEGEND/pygama data, we would want:
# gzip + shuffle + fletcher32
dset = f.create_dataset("Data",(1000,), dtype='f', compression="gzip", shuffle=True, fletcher32=True)
# (and we might also want SWMR mode, but I haven't gotten to that yet)

f.close()

## Chapter 5: Groups, Links, Iteration

REMEMBER: _Groups work mostly like dictionaries_

In [ ]:
f = h5py.File("groups.h5","w")

subg = f.create_group("subgroup")
print(subg)
print(subg.name)

# can nest
subsubg = subg.create_group("another_group")
print(subsubg.name)

# create a bunch at once
out = f.create_group('/some/big/path')
print(out)

# add some more groups w/ very simple datasets
f["Dataset1"] = 1.0
f["Dataset2"] = 2.0
f["Dataset3"] = 3.0
subg["Dataset4"] = 4.0

# access them
dset4 = f["subgroup"]["Dataset4"] # works, but inefficient
dset4 = f["subgroup/Dataset4"] # right

# use the `get` method if you're not sure if a group exists
# this is an alternative to try/except'ing everything
out = f.get("BadName")
print(out)

# take the length of the group
# (the number of objects DIRECTLY attached to the group)
print(len(f))
print(len(f["subgroup"]))

f.close()

In [ ]:
# retrieve a File object for the file in which your obj resides:
f = h5py.File("propdemo.h5","w")
grp = f.create_group("hello")
print(grp.file == f)
print(grp.parent) # return the Group obj that contains this object

# can use this to check if a file is read/write, or just get the filename
print(grp.file) # this annoyingly doesn't return a str
print(f.filename) # this does

f.close()

#### working with links

There is a LAYER between the GROUP object 
and the OBJECTS that are its members.  
The two are related by LINKS, which can be "hard", "soft", or "external".

When you assign an object to a name in a group,
that address (in memory) is recorded in the group 
and associated with the name you provided to form a link.

This means objects in an hdf5 file can have more than one name!
They can have _as many names_ as they have _links pointing to them_.
The number of links that point to an object is recorded, and when no more links exist, the space used for the object is freed.  

This kind of a link is called a _hard link_.

In [ ]:
# demonstrate the multiple-name behavior of hard links

f = h5py.File("linksdemo.h5","w")
grpx = f.create_group('x')
print(grpx.name)

# create a second link, pointing to the group
f['y'] = grpx

# retrieve an object from location /y:
grpy = f['y']
print(grpy == grpx)

# check names 
# (note, the one object in the file doesn't have a unique name!)
print(grpx.name)
print(grpy.name)

# can also create an object w/ no name
grpz = f.create_group(None)
print(grpz.name)

# there's no way to get to this group from the root group.
# if we got rid of the python object grpz, the group would
# be deleted and the space in the file reclaimed.
# to avoid this, we can link the group into the file structure
# (after creating it)
f['z'] = grpz
print(grpz.name)

# delete a link 
# when you delete the last link to an object, it's destroyed
del f['y']
del f['x']

f.close()

In [ ]:
%%bash
# if you're deleting objects inside the hdf5 file, 
# you might wind up with free space.  
# HDF5 provides the CLT "h5repack"
h5repack linksdemo.h5 repacked.h5
ls -lh linksdemo.h5 repacked.h5

In [ ]:
# soft links demo

# these store the PATH to an object

f = h5py.File("test.h5", "w")

grp = f.create_group("mygroup")
dset = grp.create_dataset("dataset", (100,))

f['hardlink'] = dset
print(f['hardlink'] == grp['dataset'])

# hard links always point to their object
grp.move('dataset', "new_dataset_name")
print(f['hardlink'] == grp['new_dataset_name'])

# move the dataset back
grp.move("new_dataset_name", "dataset")

# create a soft link that points ot the path "/mygroup/dataset"
f['softlink'] = h5py.SoftLink("/mygroup/dataset")
print(f['softlink'] == grp['dataset'])

# get the path
softlink = h5py.SoftLink('/some/path')
print(softlink.path)

# note: nothing happens on the hdf5 side until you assign a soft link
# to a name in the file
grp.move('dataset', "new_dataset_name")
dset2 = grp.create_dataset("dataset", (50,))
print(f["softlink"] == dset)
print(f["softlink"] == dset2)

# NOTE: 
# Soft links are handy if a particular dataset needs to be updated
# without breaking all the links to it elsewhere in the file

f.close()

In [ ]:
# external links demo

# NOTE: this demo is kinda weird, i get a lot of errors about
# not being able to write to files that are already open,
# when i run the cell a second time

# you can refer to objects in other files!
# might be handy when "friending" objects

f = h5py.File("file_with_resource.h5", "w")
f.create_group("mygroup")
f.close()

f2 = h5py.File("linking_file.h5", "w")
f2['linkname'] = h5py.ExternalLink("file_with_resource.h5", "mygroup")

grp = f2['linkname'] # weirdness happens here
print(grp.name)
print(grp.file)
print(f2)

# the .parent property of the retrieved object doesn't point to 
# the file where the link is, it points to the root group of the
# external file:
print(f2['/linkname'].parent == f2['/'])

f.close()
f2.close()

In [ ]:
# using hdf5's "get" to determine object types
# get, getclass, getlink

# getclass: lets you retrieve the type of an object without opening it
# getlink: lets you determine the properties of the link involved

f = h5py.File("get_demo.h5", "w")

f.create_group("subgroup") # these two are hard links
f.create_dataset("dataset", (100,))

# getclass
print("--- getclass ---")
for name in f:
    print(name, f.get(name, getclass=True))

# getlink
f["softlink"] = h5py.SoftLink("/subgroup")
with h5py.File("get_demo_ext.h5","w") as f2:
    f2.create_group("egroup")
f["extlink"] = h5py.ExternalLink("get_demo_ext.h5","/egroup")

print("\n--- getlink ---")
for name in f:
    print(name, f.get(name, getlink=True))

# note that INSTANCES of SoftLink and ExternalLink were returned,
# complete with path information.
    
# just tells you the KIND of link involved
print("\n--- both ---")
for name in f:
    print(name, f.get(name, getlink=True, getclass=True))

# note: the HardLink instance doesn't have any other properties (it's a dummy)
    
f.close()

In [ ]:
# -- 1. 
# unlike python dicts, you can't directly 
# overwrite the members of a group, to prevent data loss:

f = h5py.File("require_demo.h5","w")

f.create_group('x')
f.create_group('y')

# f.create_group('y') 
# f['y'] = f['x'] # manual hard link

# objects are immediately deleted when you unlink them from a group,
# so you have to always do it yourself:

del f['y']
f['y'] = f['x']

# -- 2.
# -- using "require_group" and "require_dataset" -- 

# if there are many datasets and groups in a file,
# it might not be appropriate to overwrite the entire file
# every time the code runs.

# so use these to check for an existing group or dataset:
# create_group -> require_group
# create_dataset -> require_dataset

f.create_dataset('dataset', (100,), dtype='i')
f.require_dataset('dataset', (100,), dtype='i')
# f.require_dataset('dataset', (100,), dtype='f') # would fail, wrong type

# require_dataset will fail if we find a conflict:
# 1. shapes don't match
# 2. requested precision (eg int64) is higher than available (eg int32)

f.close()

### Iteration and Containership

_Groups_ are actually indexed by HDF5 using a "B-tree".

They work by taking a collection of items, ordered by some scheme like string name or numeric identifier, and building a tree-like "index" to rapidly retrieve an item.

In [ ]:
# iterating over things will USUALLY be alphabetical,
# but hdf5 is really using "native" or "as fast as possible" retrieval.
# if you don't modify the group, the order doesn't change

f = h5py.File("iterationdemo.h5","w")
f.create_group('b')
f.create_group('2')
g1 = f.create_group('a')
g2 = g1.create_group('sweet')
g3 = g2.create_group('path')
f.create_dataset('data', (100,))

# these two are the same
# for key in f:
#     print(key, type(key))
# for key in f.keys():
#     print(key)

# print the subgroups of a group
for key in g1:
    print(key)

# return the actual instance
for key, val in f.items():
    print(key, val, type(val))
    
print("\n--- check for groups ---")
    
# DON'T do this:
if 'sweet' in g1: # creates a giant throwaway list
    print("yeah dog, i'm slow")

# DO this:
if 'path' in g2: # uses underlying hdf5 index
    print("totes bro")

# DO this:
if 'a/sweet/path' in f:
    print("yeah, found it!")

f.close()

In [ ]:
# NOTE: If you’re manipulating POSIX-style strings.
# and run into this problem, consider “normalizing” 
# your paths using the posixpath package: 

# note also, this cell doesn't work as written, it's just a reminder

# grp = f['/1']
# path = "../1"
# import posixpath as pp
# path = pp.normpath(pp.join(grp.name, path))
# print(path)
# if path in grp:
#     print("true")

In [ ]:
# -- multilevel iteration -- the good stuff -- 

# what if you want to iterate over every single object in the file?
# or all objects "below" a certain group?

# xtra credit: what if I want to *declare* a bunch of groups
# based on some pre-arranged list of strings (and datatypes?)

# this is done by "visitor iteration," 
# b/c hdf5 doesn't give you a nice iterable.

f = h5py.File("visit_test.h5", "w")

f.create_dataset("top_dataset", data=1.0)

f.create_group("top_group_1")
f.create_group("top_group_1/subgroup_1")
f.create_dataset("top_group_1/subgroup_1/sub_dataset_1", data=1.0)

f.create_group('top_group_2')
f.create_dataset("top_group_2/sub_dataset_2", data=1.0)

# visit : takes the object name

def printname(name):
print(name)

f.visit(printname)

# visit a subgroup only

print("\n-- visiting top group 1 only --")
grp = f["top_group_1"]
grp.visit(printname)

# get a list of every single object in the file:
mylist = []
f.visit(mylist.append)

print("\n-- Multiple links with 'visit' -- ")
# if multiple links point to your dataset, when 'visit' supplies
# a name, it may not be the one you expect:

grp['hardlink'] = f['top_group_2']
grp.visit(printname)

# the group at /top_group_2 is "mounted" in the file at
# /top_group_1/hardlink, and visit explores it correctly

del grp['hardlink']

# this tries to trick visit into visiting sub_dataset_1 twice:
print("\n -- try going to same place twice --")
grp['hardlink_to_dataset'] = grp['subgroup_1/sub_dataset_1']
grp.visit(printname)

# it fails, bc each object in a file will be visited only ONCE

# -- more visiting tricks -- 
def printobj(name, obj):
    # this returns a name and an instance,
    # we're advised to use it only when we need to check the 
    # properties of the object (like its type)
    print(name, obj)
    
print("\n-- get name and an instance --")
grp.visititems(printobj)

def print_abspath(somegroup, name):
    # print *name* as an absolute path
    # somegroup: HDF5 base group (*name* is relative to this)
    # name: object name relative to *somegroup*
    print(posixpath.join(somegroup.name, name))
    
# print the absolute path of each object in the group
print("\n -- print full paths --")
grp.visit(partial(print_abspath, grp))

# search: find a dataset that has an attribute w/ a particular value
f['top_group_2/sub_dataset_2'].attrs["special"] = 42

def find_special(name, obj):
    if obj.attrs.get('special') == 42:
        return obj

print("\n-- looking for a particular thing -- ")
out = f.visititems(find_special)
print(out)
    
f.close()

In [ ]:
# dictionary style iteration: 
# because __groups work like dictionaries!__
# https://stackoverflow.com/questions/34330283/how-to-differentiate-between-hdf5-datasets-and-groups-with-h5py
    
# this question is about differentiating GROUPS from DATASETS.
    
def ds_iterator(g, prefix=''):
    for key in g.keys():
        item = g[key]
        path = '{}/{}'.format(prefix, key)
        if isinstance(item, h5py.Dataset): # test for dataset
            yield (path, item)
        elif isinstance(item, h5py.Group): # test for group (go down)
            yield from ds_iterator(item, path)
            
with h5py.File('iterationdemo.h5', 'r') as f:
    for (path, dset) in ds_iterator(f):
        print(path, dset)

In [ ]:
# copying objects

f = h5py.File("copytest.h5","w")

f.create_group('mygroup')
f.create_group('mygroup/subgroup')
f.create_dataset("mygroup/apples", (100,))

# copying a dataset results in a brand-new dataset,
# not a reference or link 
f.copy('/mygroup/apples', '/oranges')

print(f['oranges'] == f['mygroup/apples'])

# copy() also correctly handles recursively copying groups:
print("\n-- recursive copying -- ")
f.copy('mygroup', 'mygroup2')
f.visit(printname)

# the book SAYS you can also throw Datasets directly into copy,
# but I can't get this block to work.
# I get: ValueError: Field names only allowed for compound types

print("\n -- direct ds copy --")
dset = f['/mygroup/apples']
grp = f.create_group("/testing")
print(type(dset))
# f.copy(dset, f) # copy to file (fails)
# f.copy(dset, grp) # copy to group (fails)
# f.visit(printname)

f.close()

In [ ]:
# more on object comparison

f = h5py.File("objectdemo.h5","w")

grpx = f.create_group('x')
grpy = f.create_group('y')

# simple comparison
print(grpx == f['x'])
print(grpx == grpy)

# check python's 'id' function to see if 
# the objects are ACTUALLY the same
print(id(grpx) == id(f['x']))

# but the hash is the same:
print(hash(grpx) == hash(f['x']))
      
# also note: the .file property will be equal to the group
# when we're in the root directory:
print(f == f['/'])

# check if an object is "alive" or "dead":
print(bool(grpx))
f.close()
print(bool(grpx))

## Chapter 6: Metadata and Attributes

_Attributes_ are bits of metadata you can attach to all HDF5 objects.

In [ ]:
# attribute demo

# NOTE: w/ default settings, MAX attribute size is 64 KB.

f = h5py.File("attrsdemo.h5", "w")

dset = f.create_dataset("dataset",(100,))
print(dset.attrs)

# create a new attributes
dset.attrs['title'] = "dataset from third round of experiments"
dset.attrs['sample_rate'] = 100e6 # clock freq
dset.attrs['run_id'] = 144
# dset.attrs["fail"] = {'a':1, "b":2} # nope, fails.
dset.attrs['fail'] = [1,2,"b",4] # succeeds, but is converted to np array

print(dset.attrs['title'])
print(type(dset.attrs['title'])) # gives a string
print(dset.attrs['fail'])
print(type(dset.attrs['fail']))

# iterate over attributes like a dictionary
print([x for x in dset.attrs])

# overwriting attributes is allowed!
dset.attrs["another_id"] = 42
dset.attrs["another_id"] = 100

# can also delete them:
del dset.attrs["another_id"]

# can iterate over them
print("\n--attrs--")
for name, val in dset.attrs.items():
    print(name, val)
    
# hdf5 also has a 'get' for attrs:
print(dset.attrs.get('run_id'))

# if we really needed to store a big np array as an attribute,
# we could store the data in an extra dataset, then link to it
# using the reference:

ones_dset = f.create_dataset('ones_data',data=np.ones((100,100)))
dset.attrs['ones'] = ones_dset.ref
print(dset.attrs['ones'])

# some fortran stuff can't handle variable length strings
# as attributes (like the one above), so numpy has a fixed
# length version
dset.attrs['title_fixed'] = np.string_("Another title")

# can also store unicode strings
dset.attrs["yet another title"] = u'String with accent (\u00E9)'

# if you INSIST on storing a python object, you have to pickle:
import pickle
pickled_object = pickle.dumps({'key':42}, protocol=0)
dset.attrs['object'] = pickled_object
obj = pickle.loads(dset.attrs['object'])
print(obj)

# write stuff to the file
f.flush()

f.close()

In [ ]:
%%bash
h5ls -vlr attrsdemo.h5
# some types might be "native", but they can probably be converted
# to numpy types

In [ ]:
# explicit typing
# if "native" type isn't good enough

f = h5py.File('attrs_create.h5',"w")

# specify the type of the attr
dset = f.create_dataset('dataset', (100,))
dset.attrs.create("two_byte_int", 190, dtype='i2')
dset.attrs['two_byte_int']

# for strings, you have to be more careful.

# use whatever h5py thinks you can use
dset.attrs['strings'] = ["Hello","Another string"]
print(dset.attrs['strings'])

# manually specify a variable length string
dt = h5py.special_dtype(vlen=str)
dset.attrs.create('more_strings', ["Hello","Another string"], dtype=dt)
print(dset.attrs["more_strings"])

# NOTE: the book says that these will be different datatypes, 
# but that is NOT what I'm getting.  Both are coming out as
# "variable-length null-terminated UTF-8" strings.

# modify an attr has a special function
dset.attrs.modify('two_byte_int', 33) # this might fail if the type changes
dset.attrs["two_byte_int"] = 34.0 # or you can just overwrite
print(dset.attrs['two_byte_int'])

# apparently when you modify attributes,
# you have to call flush() to get them to go into the file
f.flush()

f.close()

In [ ]:
%%bash
h5ls -vlr attrs_create.h5

## Chapter 7: Types

HDF5 supports a lot of datatypes, in some cases ones that NumPy doesn't.

In [ ]:
f = h5py.File("types_demo.h5","w")

# -- integers & floats -- 
dset = f.create_dataset('smallint', (10,), dtype=np.int8)
dset[0] = 300 # this is too big for this dtype, so it rolls over
print(dset[0])

# same thing happens in numpy:
a = np.zeros((10,), dtype=np.int8)
a[0] = 300
print(a[0]) # huh, book gets -44, I get 44.  yay versions!

# trick, a 'half-precision' float: 
# useful when a 16-bit int isn't quite enough
# i.e. your values are between 10^{-8} and 60,000
# note these are only for storage, trying to do math on them will 
# result in casting.  use Dataset.read_direct or astype
dset = f.create_dataset("half_float", (100,100,100), dtype=np.float16)
a = dset[...]
a = a.astype(np.float32)


# -- fixed length strings -- 
# NOTE: this is different from the book, b/c python3:
# see here for some more info:
# https://github.com/h5py/h5py/issues/289

dt = np.dtype("S10")
dset = f.create_dataset("fixed_string",(100,), dtype=dt)
dset[0] = "Hello".encode("utf-8") # different from book, you have to encode.
dset[0] = "thisstringhasmorethan10chars".encode("utf-8")
print(dset[0])


# -- variable length strings (use special h5py datatype) -- 
dt = h5py.special_dtype(vlen=str)
print(dt) # different from book, but is still an object
print(dt.kind) # yep, 'O' for object

dset = f.create_dataset("vlen_dataset", (100,), dtype=dt)
dset[0] = "Hello" # rad, i didn't have to encode this
dset[1] = np.string_("Hello2")
dset[3] = "X"*30 # lol, book typo?

out = dset[0]
print(out, type(out))
print(dset[0:5]) # i see the blank space from the book typo

# note, you don't always get the same type out as you put in
out = dset[0:1]
print(out.dtype) # just "object"

# NOTE: Python3 uses Unicode strings for everything!
# dt = h5py.special_dtype(vlen=str) # UTF-8 (default)
# dt = h5py.special_dtype(vlen=bytes) # ASCII 


# -- Compound types -- 
# bundle stuff together w different types, like a C struct or a table

dt = np.dtype([("temp",np.float),("pressure",np.float),("wind",np.float)])
a = np.zeros((100,), dtype=dt)

# numpy side:
# when you access a single element, you get back an object
# that supports dict-style access on the field names:
out = a['temp']
out = a[0]
print(out, out["temp"])

# hdf5 side:
dset = f.create_dataset("compound",(100,), dtype=dt)
out = dset["temp","pressure"]
print(out.shape)
print(out.dtype)
out = dset["temp", 90:100] # mix names & slices, get last 10 temp points
print(out.dtype)

# set all temps we just read to a new value and write
out[...] = 98.6
dset["temp", 90:100] = out

# complex dtypes (not in hdf5, but there is a convention)
dset = f.create_dataset("single_complex",(100,), dtype='c8')
# can check w/ h5ls to see.


# -- Enumerated types -- 
# aka. integer datatypes for which certain values are associated 
# with TEXT TAGS. 
mapping = {"red":0, "green":1, "blue":2}
dt = h5py.special_dtype(enum=(np.int8, mapping))
dset = f.create_dataset("enum",(100,), dtype=dt)

print(dset[0], dset[0].dtype) # don't get the "extras" from the enum here


# -- booleans -- 
# there isn't a native hdf5 boolean type, so h5py provides one
# can check w/ h5ls
f.create_dataset("bool",(100,), dtype=np.bool)


# -- array -- 
# good choice when you want to store multiple values of the
# SAME TYPE in a single element.
# but be careful, it's another case where 
# dset[...].dtype != dset.dtype  (i.e. you get back something else!)

dt = np.dtype("(2,2)f")
print(dt)
dset = f.create_dataset("array", (100,), dtype=dt)
print(dset.shape)
out = dset[0]
print(type(out)) # yup, just a np array now

# if we were to create a native numpy array with our type
# it would get "eaten":
a = np.zeros((100,), dtype=dt)
print(a.dtype, a.shape)

# the book says the array type is good when it's an element
# of a compound type.  for example:
# "if we had an experiment that reported an integer timestamp
# along with the output from a 2x2 light sensor, one choice
# for a data type would be:"
dt_timestamp = np.dtype('uint64')
dt_sensor = np.dtype('(2,2)f')
dt = np.dtype([ ("time",dt_timestamp), ("sensor",dt_sensor) ])

# now store and retrieve individual outputs from the experiment:
dset = f.create_dataset('mydata', (100,), dtype=dt)
dset["time",0] = time.time()

# this example from the book doesn't work. 
# I guess it's a python3 thing.
# I get this error:
# ValueError: When changing to a larger dtype, its size must be 
# a divisor of the total size in bytes of the last axis of the array.
# val = np.asarray([[1,2],[3,4]])
# print(val, type(val))
# dset["sensor",0] = val # <-- fails
# out = dset[0]
# print(out)
# print(out["sensor"])


# -- dates and times --
# notes: hdf5 doesn't ever use its "datetime" type,
# so date/time handling is "ad hoc"

# unix time (1 sec precision)
timestamp = np.dtype('u8')

# unix time (< sec precision)
print(time.time())

# ISO format:
print(datetime.datetime.now().isoformat())

# NOTE: we're not saving the time zone, daylight savings, or anything
# else here.  so be careful!  maybe use attrs if you need to.

f.close()

## Chapter 8: References, Types, and Dimension Scales

"Some of the best features in HDF5 are those that help you to express _relationships_ between pieces of your data."

3 main things:
- _References_ (HDF5's pointer type) are a great way to store _links to objects_ as data.
- _Named types_ let you enforce type consistency across datasets.
- _Dimension Scales_ (an HDF5 standard), let you attach _physically meaningful axes_ to your data

In [ ]:
# object references

# links in a group serve to locate objects.
# REFERENCES are another mechanism to do this,
# and they can be stored AS DATA, in attrs and datasets.

f = h5py.File("refs_demo.h5","w")

grp1 = f.create_group("group1")
grp2 = f.create_group("group2")
dset = f.create_dataset("mydata", shape=(100,))

# get an object's reference (the pointer to the object in the file)
print(grp1.ref)

# dereference
out = f[grp1.ref]
print(out == grp1)

# check instance
print(isinstance(grp1.ref, h5py.Reference))

# refs are local to their file
# with h5py.File("anotherfile.h5","w") as f2:
#     out = f2[grp1.ref] # ValueError: unable dereference object
      
# Unlike links, references can be stored as data:
# they are "unbreakable links."

grp1.attrs['dataset'] = dset.name
out = f[grp1.attrs['dataset']]
print(out == dset)

# if we rename the dataset, the above will break:
f.move("mydata","mydata2")
# out = f[grp1.attrs['dataset']] # KeyError: "unable to open object"

# instead, use an object reference
grp1.attrs['dataset'] = dset.ref
out = f[grp1.attrs['dataset']]
print(out == dset) # this time, True

# NOTE:
# when you open an object by dereferencing, every now and then it's 
# possible HDF5 won't be able to figure out the object's name.
# In that case, obj.name will return None.

# storing a reference as data requires a special hdf5 datatype:
dt = h5py.special_dtype(ref=h5py.Reference)
# print(dt.kind) # "O" for object

ref_dset = f.create_dataset("references",(10,), dtype=dt)
out = ref_dset[0]
print(out)

# derefrence the null reference: you get a ValueError
# print(f[out])

# check for a null reference with a bool
print(bool(out), bool(grp1.ref))

f.close()

In [ ]:
# region references

# "one of the coolest features of HDF5"

# maybe you want to store a region of interest (ROI) of a dataset,
# so during later analysis you don't have to process the whole thing!

# they store a slice argument for later use!

f = h5py.File('refs2.h5','w')

dt = h5py.special_dtype(ref=h5py.Reference)
# dset = f.create_dataset("references",(10,), dtype=dt)
dset = f.create_dataset('mydata', shape=(100,))

print(dset.name)
print(dset.shape)
print(dset.regionref) # <-- this thing

# create a region ref
ref_out = dset.regionref[10:90]
print(ref_out)

# the book says these exist, but they don't work for me
# dset.regionref.shape(ref_out)(100,) # same as parent
# dset.regionref.selection(ref_out)(80,) # shape of selection

# if you have a region reference (you read from a file, presumably)
# you can use it to slice the dataset on retrieval:
data = dset[ref_out]

print(data)

f.close()

In [ ]:
# fancy indexing

# methods like Boolean arrays will always have a 1-D shape

f = h5py.File("fancy_indexing.h5","w")

# numpy side
arr = np.random.random((3,3))
idx = np.where(arr > 0.5) # returns a tuple, only indexes
print(type(idx),"\n",idx) 

# hdf5 side
dset_random = f.create_dataset("small_example",(3,3))
dset_random[...] = arr
idx = dset_random[...] > 0.5 # returns a boolean array, same shape as orig
print(type(idx),"\n", idx)

# create a region reference from this selection:
random_ref = dset_random.regionref[idx]

# access it
print(dset_random.regionref.selection(random_ref))

data = dset_random[random_ref]
print(data)

# huhhhh, this returns a 1d array, not a (3,3).

# the book says the list-based selection is returned as a 1D
# array, following "C order": the selection avances through the last
# index, then the next to last, and so on.
# "this is a limitation of the HDF5 library."

# last trick: if you have a region reference, you can use
# it as an OBJECT reference to retrieve the whole dataset:

# imagine you didn't have this, but just the file and a regionref
dset = f.create_dataset('mydata', shape=(100,)) 

ref_out = dset.regionref[10:90]

print(f[ref_out])

# and the actual subset:
selected_data = f[ref_out][ref_out]


f.close()

In [3]:
# named types

# imagine you had many datasets in a file (like a bunch of images),
# and you wanted to be sure every image has exactly the same type.
# HDF5 lets you save the datatype to the FILE.
# when you call create_dataset, you supply the stored type
# and HDF5 will "link" the type to the brand new dataset.

f = h5py.File("named_types.h5","w")

f["mytype"] = np.dtype('float32')

out = f['mytype']
out.attrs['info'] = "this is my datatype"

print(out)
print(out.dtype)
print(out.name)
print(out.parent)

# linking to named types

dset = f.create_dataset("typedemo",(100,), dtype=f['mytype']) # a link 

# for attributes, you have to supply the type with create()
f.attrs.create("attribute_demo", 1.0, dtype=f['mytype'])

# you can't modify named types.  and if you unlink them, they still
# stick around until every object they're associated with is deleted.
del f['mytype']
f['mytype'] = np.dtype('int16')
dset = f['typedemo']
print(dset.dtype) # <-- nope, still float32.

f.close()

<HDF5 named type "mytype" (dtype <f4)>
float32
/mytype
<HDF5 group "/" (1 members)>
float32


In [7]:
# dimension scales

# in the real world, we have units attached to things!

f = h5py.File("dim_scales.h5","w")

dset = f.create_dataset('temperatores', (100,100,100), dtype='f')

# we might start by trying to store important info with the attrs:
dset.attrs["temp_units"] = "C"
dset.attrs["steps"] = [10000,10000,100] # measurement resolution, say
dset.attrs["axes"] = ["x", "y", "z"]

# but that's chump change.  
# HDF5 has "DIMENSION SCALES", which you can read instead:

for name in dset.attrs:
    del dset.attrs[name]

print(dset.dims) # <-- entry point to a separate dataset w/ metadata

f.create_dataset("scale_x", data=np.arange(100)*10e3)
f.create_dataset("scale_y", data=np.arange(100)*10e3)
f.create_dataset("scale_z", data=np.arange(100)*100.0)

dset.dims.create_scale(f['scale_x'], "Simulation X (North) axis")
dset.dims.create_scale(f['scale_y'], "Simulation Y (East) axis")
dset.dims.create_scale(f['scale_z'], "Simulation Z (Vertical) axis")

for key, val in f['scale_x'].attrs.items():
    print(key, " : ", val)

# huh.  all it did is just attach a few attributes w/ 
# standardized names and values.

# now let's associate the scales to the dataset.

dset.dims[0].attach_scale(f['scale_x'])
dset.dims[1].attach_scale(f['scale_y'])
dset.dims[2].attach_scale(f['scale_z'])

dset.dims[0].label = 'x'
dset.dims[1].label = 'y'
dset.dims[2].label = 'z'

# dims[N] is a proxy keeping track of which dimension scales are
# attached to the first axis of the dataset
# people who create plots w/ an axis on every side can use multiple scales

print(dset.dims[0][0])
print(dset.dims[0][0][...]) # <-- get the actual axis values

# so by saving the actual axis values, we can actually handle 
# variable length binning and other stuff that just using "attrs"
# wouldn't have allowed us to do!

f.close()

<Dimensions of HDF5 object at 4536618856>
CLASS  :  b'DIMENSION_SCALE'
NAME  :  b'Simulation X (North) axis'
<HDF5 dataset "scale_x": shape (100,), type "<f8">
[     0.  10000.  20000.  30000.  40000.  50000.  60000.  70000.  80000.
  90000. 100000. 110000. 120000. 130000. 140000. 150000. 160000. 170000.
 180000. 190000. 200000. 210000. 220000. 230000. 240000. 250000. 260000.
 270000. 280000. 290000. 300000. 310000. 320000. 330000. 340000. 350000.
 360000. 370000. 380000. 390000. 400000. 410000. 420000. 430000. 440000.
 450000. 460000. 470000. 480000. 490000. 500000. 510000. 520000. 530000.
 540000. 550000. 560000. 570000. 580000. 590000. 600000. 610000. 620000.
 630000. 640000. 650000. 660000. 670000. 680000. 690000. 700000. 710000.
 720000. 730000. 740000. 750000. 760000. 770000. 780000. 790000. 800000.
 810000. 820000. 830000. 840000. 850000. 860000. 870000. 880000. 890000.
 900000. 910000. 920000. 930000. 940000. 950000. 960000. 970000. 980000.
 990000.]


## Chapter 9: Parallelizing HDF5, aka "Concurrency"

Python includes a single master lock that governs access to the interpreter’s functions, called the Global Interpreter Lock or GIL. This lock serializes access from multiple threads to basic resources like object reference counting. You can have as many threads as you like in a Python program, but only one at a time can use the interpreter.

Access to HDF5 is serialized using locking, so only _one thread at a time_ can work with the library.

`h5py` is _thread-safe_, in that you can safely share objects between threads without corruption, and there's no global state that lets one thread stomp on another.

NOTE: The book says there are a bunch of problems with `multiprocessing` with HDF5 as of October 2013.  I went to the h5py github and downloaded an example to this folder, `multiprocessing_example.py`, that runs beautifully and displays a great fractal.  If you really need to be sure you're doing it right, check both.

The issue with `multiprocessing` is that _new processes inherit the state of the HDF5 library from the parent process_.  You end up with multiple processes "fighting" each other for the same file.  Some things to try:

1. Do all your file I/O in the main process, but don’t have files open when you invoke the multiprocessing features.  
2. Multiple subprocesses can safely read from the same file, but only open it once the new process has been created.  
3. Have each subprocess write to a different file, and merge them when finished.

Collette, Andrew. Python and HDF5: Unlocking Scientific Data . O'Reilly Media. Kindle Edition. 

In [16]:
# first example : `threading` module
# 
# create a single shared HDF5 file and two threads that do computation,
# and write to the file.  Access is managed with `threading.RLock`.

import threading
import random

f = h5py.File("thread_demo.h5", "w")
dset = f.create_dataset("data",(2,1024),dtype='f')

lock = threading.RLock()

class ComputeThread(threading.Thread):
    def __init__(self, axis):
        # one thread does dset[0,:], the other dset[1,:]
        self.axis = axis 
        threading.Thread.__init__(self)
        
    def run(self):
        # perform a series of (simulated) computations 
        # and save to dataset.
        for idx in range(1024):
            rando_number = random.random()*0.01
            time.sleep(rando_number) # do computation

            # magic step
            with lock:
                dset[self.axis, idx] = rando_number # save to dataset
                
thread1 = ComputeThread(0)
thread2 = ComputeThread(1)

print("Running fake data taking ...")
thread1.start()
thread2.start()

# wait until both threads have finished
thread1.join()
thread2.join()

print("huzzah")

f.close()
        

Running fake data taking ...
huzzah


In [13]:
# second example:
# using the `multiprocessing` module

# distribute the work among worker processes

from multiprocessing import Pool

# a quick first example
p = Pool(2) # creates a 2-process pool
words_in = ['hello','some','words']
words_out = p.map(str.upper, words_in) # <- call the workers w/ a function
print(words_out, type(words_out))

# -------
# suppose we have a file containing a 1d dataset of coordinate pairs,
# and we need to compute their distance from the origin.
# this is good to parallelize because each computation doesn't
# depend on the others.

with h5py.File('coords.h5','w') as f:
    dset = f.create_dataset('coords',(1000,2), dtype='f4')
    dset[...] = np.random.random((1000,2))
    
def distance(arr):
    # compute distance from origin to the point
    # arr is a shape (2,) array
    return np.sqrt(np.sum(arr**2))

# Load data and close the input file
with h5py.File('coords.h5', 'r') as f:
    data = f['coords'][...]

# run in "parallel", but the file IO is explicitly in the main process
p = Pool(4)
result = np.array(p.map(distance, data))

with h5py.File("coords.h5") as f:
    f['distances'] = result
    
# "doing anything more complex with multiprocessing and HDF5
# gets complicated. Your process can't all access the same file."

# -- I'm going to skip the second example here, which
# just calclulates the same thing with blocks of 100 numbers
# instead of single values, spits them all to different files,
# and recombines.  ugh, gross.  

['HELLO', 'SOME', 'WORDS'] <class 'list'>


### MPI and Parallel HDF5

"What if there were a way to share a single file between processes, automatically synchronizing reads and writes?  We would get around the limitations on passing open files to child processes.

MPI-based applications work by launching _multiple parallel instances of the Python interpreter_.  The key difference compared to `multiprocessing` is that the processes are _peers_, unlike the child processes used for `Pool`.

All file access has to be coordinated though the MPI library as well. If not, multiple processes would “fight” over the same file on disk. Thankfully, HDF5 itself handles nearly all the details involved with this. All you need to do is open shared files with a special driver, and follow some constraints for data consistency.

For more examples, look at http://mpi4py.scipy.org/

Collette, Andrew. Python and HDF5: Unlocking Scientific Data . O'Reilly Media. Kindle Edition. 

In [14]:
# NOTE: to install prereqs, I just did:
# $ brew install mpich
# $ pip install mpi4py

# i can come back to this later, if i feel like it

## Appendix A: Single-Write, Multiple Reader Mode (SWMR)

This isn't in the Kindle version of the book I got, but it's an important new feature.

Now I'll work off of the docs:

http://docs.h5py.org/en/stable/swmr.html


## Appendix B: Virtual Datasets (VDS)

Working off the docs here too:

http://docs.h5py.org/en/stable/vds.html